# Parcial 3 Métodos Númericos

__Valentina Galindo Peña__

*__ID__*: 000484044

__Juan David Parra Cantor__

*__ID__*: 000324320

## Tema B

Se plantea la ecuación:

$$
y'' = 2y' - y + xe^x - x
$$

Con condiciones de 

- y(0) = 0
- y(2) = - 4
- 0 $\le$ x $\le$ 2


Y en el ejercicio se da la solución exacta:

$$
y(x) = \frac{1}{6}(x^3e^x) - \frac{5}{3}(xe^x) + 2e^x - x -2
$$
